In [1]:
from pyscal_rdf import StructureGraph
from pyscal_rdf.visualize import visualize_sample
from rdflib import URIRef, Namespace, BNode, Literal, XSD, RDF, Graph

In [2]:
CMSO = Namespace("https://purls.helmholtz-metadaten.de/cmso/")

In [3]:
g = StructureGraph()

In [4]:
struct_Fe = g.create_element("Fe")

Fe


In [6]:
struct_Fe = g.create_element("Si")

Si


In [7]:
g.n_samples

2

In [6]:
c = samples[0][0]
c

rdflib.term.BNode('Ne8a9e5eee4b84b10ba91c6e1fa3138a4')

In [7]:
g.visualise(sample=c)

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'avoidOverlap': True, 'animate': True}, cytoscape_style=[{'s…